In [1]:
import pandas as pd
import glob
import numpy as np
from tqdm import tqdm
#from dask.dataframe import from_pandas
#import dask.dataframe as dd

In [2]:
def data_frame_adder(temp_frame,file_name):
    file_name=file_name.split('.csv')[0]
    temp_frame['file_name']=file_name
    temp_frame['subject']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[0]))
    temp_frame['session']=temp_frame['file_name'].apply(lambda x: int(x.split('subject_')[1].split('_')[1]))
    #temp_frame['type']=temp_frame['file_name'].apply(lambda x: x.split('__')[1])
    temp_frame = temp_frame.drop(columns=['file_name'])
    temp_frame = temp_frame[temp_frame.columns[::-1]]
    temp_frame = temp_frame.sort_values(by=['session','subject'], axis=0)
    col_list1 = list(temp_frame.columns)[0:2]
    col_list2 = list(temp_frame.columns)[2:]
    col_list2.sort()
    col_list=col_list1+col_list2
    
    #temp_frame = swap_columns(temp_frame, 'subject', 'session')
    # Swapping the columns
    temp_frame = temp_frame.reindex(columns=col_list)

    return temp_frame

In [3]:
Training_data_files=glob.glob('TrainingData/*.csv')

In [4]:
file_name_list_x=[]
file_name_list_x_time=[]
file_name_list_y=[]
file_name_list_y_time=[]
for file_name in tqdm(Training_data_files):
    if 'x.csv' in file_name:
        file_name_list_x.append(file_name)
    elif 'x_time.csv' in file_name:
        file_name_list_x_time.append(file_name)
    elif 'y.csv' in file_name:
        file_name_list_y.append(file_name)
    else:
        file_name_list_y_time.append(file_name)
file_name_list_x.sort()
file_name_list_x_time.sort()
file_name_list_y.sort()
file_name_list_y_time.sort()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 116/116 [00:00<00:00, 921475.88it/s]


In [5]:
Subject_list= [f.split('_')[1] for f in file_name_list_x]
Session_list= [f.split('_')[2] for f in file_name_list_x]

In [195]:
x_sensor_dat=pd.DataFrame([])
y_sensor_dat=pd.DataFrame([])
x_time_dat=[]
for sub,sess in zip(Subject_list,Session_list):
    #x_sensor_dat=pd.DataFrame(x_sensor_dat.append(pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x.csv",header=None)))
    
    x_sensor_dat=pd.concat([pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x.csv",header=None),pd.read_csv(f"TrainingData/subject_{sub}_{sess}__x_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1)

    x_sensor_dat['measurements']=x_sensor_dat.apply(lambda x: [x[0],x[1],x[2],x[3]],axis=1)
    x_sensor_dat=x_sensor_dat[['time_stamp','measurements']]
    x_sensor_dat["time_stamp"]=pd.to_datetime(x_sensor_dat["time_stamp"],unit='s').round('us')
    #x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    x_sensor_dat['Subject']=int(sub)
    x_sensor_dat['Session']=int(sess)
    #x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    
    y_sensor_dat=pd.concat([pd.read_csv(f"TrainingData/subject_{sub}_{sess}__y.csv",header=None),pd.read_csv(f"TrainingData/subject_{sub}_{sess}__y_time.csv",header=None).rename(columns={0:'time_stamp'})],axis=1).rename(columns={0:'labels'})
    #y_sensor_dat['labels']=y_sensor_dat.apply(lambda y: [y[0],y[1],y[2],y[3]],axis=1)
    y_sensor_dat=y_sensor_dat[['time_stamp','labels']]
    y_sensor_dat["time_stamp"]=pd.to_datetime(y_sensor_dat["time_stamp"],unit='s').round('ms')
    trp=pd.concat([pd.DataFrame({'Subject':sub},index=[x_sensor_dat.index[-1]]),pd.DataFrame({'Session':sess},index=[x_sensor_dat.index[-1]]),pd.DataFrame({'labels':0},index=[x_sensor_dat.index[-1]]),pd.DataFrame({'time_stamp':np.max(x_sensor_dat['time_stamp'])},index=[x_sensor_dat.index[-1]])],axis=1)
    y_sensor_dat=pd.concat([y_sensor_dat,trp],axis=0)
    trp=pd.concat([pd.DataFrame({'Subject':sub},index=[x_sensor_dat.index[0]]),pd.DataFrame({'Session':sess},index=[x_sensor_dat.index[0]]),pd.DataFrame({'labels':0},index=[x_sensor_dat.index[0]]),pd.DataFrame({'time_stamp':np.min(x_sensor_dat['time_stamp'])},index=[x_sensor_dat.index[0]])],axis=1)
    y_sensor_dat=pd.concat([trp,y_sensor_dat],axis=0)
    # y_sensor_dat=y_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    y_sensor_dat['Subject']=int(sub)
    y_sensor_dat['Session']=int(sess)
    x_sensor_dat=x_sensor_dat.set_index('time_stamp').asfreq('0.025S')
    y_sensor_dat=y_sensor_dat.set_index('time_stamp').asfreq(freq='0.025S', method='bfill')
                                        #.resample('0.025S').interpolate(method='linear')
    x_y_dat=pd.merge(x_sensor_dat,y_sensor_dat, how='inner', left_index=True, right_index=True)
    break

In [196]:
x_y_dat

,measurements,Subject_x,Session_x,Subject_y,Session_y,labels
time_stamp,,,,,,
1970-01-01 00:00:00.000,"[4.435275, 8.196063, 2.974488, 0.01421523]",1,1,1,1,0
1970-01-01 00:00:00.025,"[4.18692, 8.344455, 2.908057, 0.005770759]",1,1,1,1,0
1970-01-01 00:00:00.050,"[4.544637, 8.408659, 2.89, 0.007967429]",1,1,1,1,0
1970-01-01 00:00:00.075,"[4.849308, 8.411614, 2.900692, 0.02777778]",1,1,1,1,0
1970-01-01 00:00:00.100,"[4.50919, 8.118649, 2.847298, 0.02157696]",1,1,1,1,0
...,...,...,...,...,...,...
1970-01-01 00:15:47.125,"[1.927349, 9.584698, 0.94, -0.001852465]",1,1,1,1,0
1970-01-01 00:15:47.150,"[1.93, 9.562658, 0.8746842, 0.003927394]",1,1,1,1,0
1970-01-01 00:15:47.175,"[1.906144, 9.555361, 0.8661444, -0.001111111]",1,1,1,1,0


In [185]:
x_sensor_dat

,measurements,Subject,Session
time_stamp,,,
1970-01-01 00:00:00.000,"[4.435275, 8.196063, 2.974488, 0.01421523]",1,1
1970-01-01 00:00:00.025,"[4.18692, 8.344455, 2.908057, 0.005770759]",1,1
1970-01-01 00:00:00.050,"[4.544637, 8.408659, 2.89, 0.007967429]",1,1
1970-01-01 00:00:00.075,"[4.849308, 8.411614, 2.900692, 0.02777778]",1,1
1970-01-01 00:00:00.100,"[4.50919, 8.118649, 2.847298, 0.02157696]",1,1
...,...,...,...
1970-01-01 00:15:47.125,"[1.927349, 9.584698, 0.94, -0.001852465]",1,1
1970-01-01 00:15:47.150,"[1.93, 9.562658, 0.8746842, 0.003927394]",1,1
1970-01-01 00:15:47.175,"[1.906144, 9.555361, 0.8661444, -0.001111111]",1,1


In [186]:
y_sensor_dat

,Subject,Session,labels
time_stamp,,,
1970-01-01 00:00:00.000,1,1,0
1970-01-01 00:00:00.025,1,1,0
1970-01-01 00:00:00.050,1,1,0
1970-01-01 00:00:00.075,1,1,0
1970-01-01 00:00:00.100,1,1,0
...,...,...,...
1970-01-01 00:15:47.125,1,1,0
1970-01-01 00:15:47.150,1,1,0
1970-01-01 00:15:47.175,1,1,0


In [183]:
y_sensor_datlast

NameError: name 'y_sensor_datlast' is not defined

In [68]:
[pd.DataFrame({'time_stamp':time_stamp},pd.DataFrame({'Subject':sub},
                                                                   index=[x_sensor_dat.index[-1]]),
                            pd.DataFrame({'Session':sess},
                                         index=[x_sensor_dat.index[-1]]),
                            pd.DataFrame({'labels':np.nan},i
                                         ndex=[x_sensor_dat.index[-1]])]

SyntaxError: '[' was never closed (1955934303.py, line 1)

In [65]:
trp=pd.concat(,axis=1)
               
               

SyntaxError: '[' was never closed (752780160.py, line 1)

In [58]:
y_sensor_dat

,Subject,Session,labels
1970-01-01 00:00:00.000,001,01,NaN
1970-01-01 00:00:00.020,001,01,0.0
1970-01-01 00:00:00.045,001,01,NaN
1970-01-01 00:00:00.070,001,01,NaN
1970-01-01 00:00:00.095,001,01,NaN
...,...,...,...
1970-01-01 00:15:47.145,001,01,NaN
1970-01-01 00:15:47.170,001,01,NaN
1970-01-01 00:15:47.195,001,01,NaN
1970-01-01 00:15:47.220,001,01,0.0


In [44]:
y_sensor_dat.asfreq(freq='0.025S')

,Subject,Session,labels
1970-01-01 00:00:00.000,001,01,NaN
1970-01-01 00:00:00.025,NaN,NaN,NaN
1970-01-01 00:00:00.050,NaN,NaN,NaN
1970-01-01 00:00:00.075,NaN,NaN,NaN
1970-01-01 00:00:00.100,NaN,NaN,NaN
...,...,...,...
1970-01-01 00:15:47.125,NaN,NaN,NaN
1970-01-01 00:15:47.150,NaN,NaN,NaN
1970-01-01 00:15:47.175,NaN,NaN,NaN
1970-01-01 00:15:47.200,NaN,NaN,NaN


In [ ]:
x_sensor_dat

In [ ]:
y_sensor_dat.index[1]-y_sensor_dat.index[0]

In [ ]:
x_sensor_dat.index[1]-x_sensor_dat.index[0]

In [ ]:

data_frame_x=pd.DataFrame([])
data_frame_x_time=pd.DataFrame([])
data_frame_y=pd.DataFrame([])
data_frame_y_time=pd.DataFrame([])
for file in tqdm(Training_data_files):
    file_name=file.split('/')[-1]
    temp_frame=pd.read_csv(file,header=None)
    if 'x.csv' in file_name:
        data_frame_x=pd.concat([data_frame_x,data_frame_adder(temp_frame,file_name)],axis=0,ignore_index=1).sort_values(by=['session','subject'], axis=0)
        
    elif 'x_time.csv' in file_name:
        data_frame_x_time=pd.concat([data_frame_x_time,data_frame_adder(temp_frame,file_name)],axis=0,ignore_index=1).sort_values(by=['session','subject'], axis=0)
        
    elif 'y.csv' in file_name:
        data_frame_y=pd.concat([data_frame_y,data_frame_adder(temp_frame,file_name)],axis=0,ignore_index=1).sort_values(by=['session','subject'], axis=0)
        
    else:
        data_frame_y_time=pd.concat([data_frame_y_time,data_frame_adder(temp_frame,file_name)],axis=0,ignore_index=1).sort_values(by=['session','subject'], axis=0)

data_frame_x_time=data_frame_x_time.rename(columns={0:'time_stamp'})[['time_stamp']]
data_frame_y=data_frame_y.rename(columns={0:'labels'})
data_frame_y_time=data_frame_y_time.rename(columns={0:'time_stamp'})[['time_stamp']]




In [ ]:
total_x

In [ ]:
pandas.DatetimeIndex

In [ ]:
total_dict={}

In [ ]:
total_dict[0]={}

In [ ]:
total_x=pd.concat([data_frame_x_time,data_frame_x],axis=1)
subject=total_x.subject.unique()
sessions=total_x.session.unique()
#total_x["time_stamp"]=pd.to_datetime(total_x["time_stamp"],unit='s').round('us')
#total_x=total_x.set_index(['time_stamp'])

In [ ]:
total_dict=pd.DataFrame({'subject':subject,'sessions':sessions})

In [ ]:
total_dict={'subject':list(subject)}

In [ ]:
total_dict={}

In [ ]:
total_dict['subject']={tuple(subject)}

In [ ]:
total_dict=pd.DataFrame(np.zeros((len(subject),len(sessions))))

In [ ]:
sess

In [ ]:
sub

In [ ]:


#total_x=total_x.set_index(['subject','session'])
#total_x["time_stamp"]=pd.to_datetime(total_x["time_stamp"],unit='s')
#.asfreq('0.025S')
# total_x=total_x.set_index(['subject','session','time_stamp'])
sub_list=[]
for sub in subject:
    sess_list=[]
    for sess in sessions:
        #P=pd.DataFrame([])
        # P['subject']=sub
        # P['sessions']=sess
        P=total_x[(total_x.subject==sub)&(total_x.session==sess)].asfreq('0.025S')
        P['subject']=sub
        P['sessions']=sess
        sess_list.append(P)
    sub_list.append(sess_list)

# total_y=pd.concat([data_frame_y_time,data_frame_y],axis=1)
# total_y["time_stamp"]=pd.to_datetime(total_y["time_stamp"],unit='s')
# total_y=total_y.set_index('time_stamp')

In [ ]:
total_x[(total_x.subject==sub)&(total_x.session==sess)]

In [ ]:
t=total_x[(total_x.subject==sub)&(total_x.session==sess)]

In [ ]:
t[t.time_stamp.duplicated()]

In [ ]:
r

In [ ]:
t[t['time_stamp']==1030.775]

In [ ]:
len(t)

In [ ]:
r=pd.read_csv('TrainingData/subject_001_02__x_time.csv',header=None)

In [ ]:
r1=pd.read_csv('TrainingData/subject_001_02__x.csv',header=None)

In [ ]:
r1.loc[41231]

In [ ]:
r1

In [ ]:
len(r)

In [ ]:
total_y_updated=total_y.copy()

In [ ]:
total_y_updated['time_stamp']=pd.to_timedelta(total_y[(total_y.session==1)&(total_y.subject==1)]['time_stamp'])

In [ ]:
total_x[(total_x.session==2)&(total_x.subject==2)]

In [ ]:
total_y.resample('30S').ffill()

In [ ]:
total_x.index[0]

In [ ]:
total_x.index[1]-total_x.index[0]

In [ ]:
(total_x.index[1]-total_x.index[0]).total_seconds()

In [ ]:
total_x.index=pd.DatetimeIndex(total_x.index)